# Getting started with the Azure Agent Service with the OpenAI SDK

In [4]:
!pip install azure-ai-projects
!pip install azure-identity


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [23]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

True

In [24]:
print(os.getenv("PROJECT_CONNECTION_STRING"))
print(os.getenv("BING_CONNECTION_NAME"))

eastus2.api.azureml.ms;3490bb6d-9e8c-46c5-b635-f8fea8df3d69;agent-quickstart-rg2;project-demo-agents-oaic
bing_grounding


In [3]:
import os, time
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from openai import AzureOpenAI

Now let's create an agent

In [10]:
with AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
) as project_client:

    # Explicit type hinting for IntelliSense
    client: AzureOpenAI = project_client.inference.get_azure_openai_client(
        # The latest API version is 2024-10-01-preview
        api_version = os.environ.get("AZURE_OPENAI_API_VERSION"),
    )

    with client:
        agent = client.beta.assistants.create(
            model="gpt-4o-mini", name="my-agent", instructions="You are a helpful agent"
        )
        print(f"Created agent, agent ID: {agent.id}")

        thread = client.beta.threads.create()
        print(f"Created thread, thread ID: {thread.id}")

        message = client.beta.threads.messages.create(thread_id=thread.id, role="user", content="Hello, tell me a joke")
        print(f"Created message, message ID: {message.id}")

        run = client.beta.threads.runs.create(thread_id=thread.id, assistant_id=agent.id)

        # Poll the run while run status is queued or in progress
        while run.status in ["queued", "in_progress", "requires_action"]:
            time.sleep(1)  # Wait for a second
            run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
            print(f"Run status: {run.status}")

        client.beta.assistants.delete(agent.id)
        print("Deleted agent")

        messages = client.beta.threads.messages.list(thread_id=thread.id)
        print(f"Messages: {messages}")

AuthenticationError: Error code: 401 - {'error': {'code': 'PermissionDenied', 'message': 'Principal does not have access to API/Operation.'}}

# Using other Models with the Azure Agent Service

# Getting started with the Azure AI Foundry SDK

In [29]:
import os
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import CodeInterpreterTool
from azure.identity import DefaultAzureCredential
from typing import Any
from pathlib import Path

print(os.getenv("PROJECT_CONNECTION_STRING"))
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=os.environ["PROJECT_CONNECTION_STRING"]
)

with project_client:
    # Create an instance of the CodeInterpreterTool
    code_interpreter = CodeInterpreterTool()

    # The CodeInterpreterTool needs to be included in creation of the agent
    agent = project_client.agents.create_agent(
        model="gpt-4o-mini",
        name="my-agent",
        instructions="You are helpful agent",
        tools=code_interpreter.definitions,
        tool_resources=code_interpreter.resources,
    )
    print(f"Created agent, agent ID: {agent.id}")

    # Create a thread
    thread = project_client.agents.create_thread()
    print(f"Created thread, thread ID: {thread.id}")

    # Create a message
    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content="Could you please create a bar chart for the operating profit using the following data and provide the file to me? Company A: $1.2 million, Company B: $2.5 million, Company C: $3.0 million, Company D: $1.8 million",
    )
    print(f"Created message, message ID: {message.id}")

    # Run the agent
    run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
    print(f"Run finished with status: {run.status}")
    print(f"Run ID: {run.id}")

    if run.status == "failed":
        # Check if you got "Rate limit is exceeded.", then you want to get more quota
        print(f"Run failed: {run.last_error}")

    # Get messages from the thread
    messages = project_client.agents.list_messages(thread_id=thread.id)
    print(f"Messages: {messages}")

    # Get the last message from the sender
    last_msg = messages.get_last_text_message_by_sender("assistant")
    if last_msg:
        print(f"Last Message: {last_msg.text.value}")

    # Generate an image file for the bar chart
    for image_content in messages.image_contents:
        print(f"Image File ID: {image_content.image_file.file_id}")
        file_name = f"{image_content.image_file.file_id}_image_file.png"
        project_client.agents.save_file(file_id=image_content.image_file.file_id, file_name=file_name)
        print(f"Saved image file to: {Path.cwd() / file_name}")

    # Print the file path(s) from the messages
    for file_path_annotation in messages.file_path_annotations:
        print(f"File Paths:")
        print(f"Type: {file_path_annotation.type}")
        print(f"Text: {file_path_annotation.text}")
        print(f"File ID: {file_path_annotation.file_path.file_id}")
        print(f"Start Index: {file_path_annotation.start_index}")
        print(f"End Index: {file_path_annotation.end_index}")
        project_client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=Path(file_path_annotation.text).name)

    # Delete the agent once done
    project_client.agents.delete_agent(agent.id)
    print("Deleted agent")

eastus2.api.azureml.ms;3490bb6d-9e8c-46c5-b635-f8fea8df3d69;agent-quickstart-rg2;project-demo-agents-oaic
Created agent, agent ID: asst_8G2t9MSmdRp7rDrPGHLjoe23
Created thread, thread ID: thread_bfbgqDBKcdYwhqoY9Uj0nLg1
Created message, message ID: msg_JZavUgTnBVv2PtTDmdQrOubJ


KeyboardInterrupt: 

In [8]:
print(os.getenv("PROJECT_CONNECTION_STRING"))


eastus2.api.azureml.ms;3490bb6d-9e8c-46c5-b635-f8fea8df3d69;agent-quickstart-rg2;project-demo-agents-oaic


In [30]:
# Import required libraries
import os
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import CodeInterpreterTool
from azure.identity import DefaultAzureCredential
from typing import Any
from pathlib import Path

# Explanation: These imports are needed to work with Azure's AI Project Client,
# handle credentials, and manage file paths.


In [31]:
# Initialize the AIProjectClient
connection_string = os.environ["PROJECT_CONNECTION_STRING"]
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=connection_string
)

# Open the client context
project_client.__enter__()

In [32]:
code_interpreter = CodeInterpreterTool()

# The CodeInterpreterTool needs to be included in creation of the agent
agent = project_client.agents.create_agent(
    model="gpt-4o-mini",
    name="my-agent",
    instructions="You are helpful agent",
    tools=code_interpreter.definitions,
    tool_resources=code_interpreter.resources,
)
print(f"Created agent, agent ID: {agent.id}")

Created agent, agent ID: asst_elly6NfbHNtVk328Xza5ToMD


In [33]:
thread = project_client.agents.create_thread()
print(f"Created thread, thread ID: {thread.id}")

# Create a message
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="Could you please create a bar chart for the operating profit using the following data and provide the file to me? Company A: $1.2 million, Company B: $2.5 million, Company C: $3.0 million, Company D: $1.8 million",
)
print(f"Created message, message ID: {message.id}")
# Process the run with the agent


Created thread, thread ID: thread_ugbFNnWyYHWbjDlanNcmNuPR
Created message, message ID: msg_YRO7MYwDdT2koReZOaj4g2uS


In [34]:
run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
print(f"Run finished with status: {run.status}")
print(f"Run ID: {run.id}")
if run.status == "failed":
    # Handle failure case
    print(f"Run failed: {run.last_error}")



KeyboardInterrupt: 

In [12]:
# Get messages from the thread
messages = project_client.agents.list_messages(thread_id=thread.id)
print(f"Messages: {messages}")

# Get the last message from the sender
last_msg = messages.get_last_text_message_by_sender("assistant")
if last_msg:
    print(f"Last Message: {last_msg.text.value}")

# Generate an image file for the bar chart
for image_content in messages.image_contents:
    print(f"Image File ID: {image_content.image_file.file_id}")
    file_name = f"{image_content.image_file.file_id}_image_file.png"
    project_client.agents.save_file(file_id=image_content.image_file.file_id, file_name=file_name)
    print(f"Saved image file to: {Path.cwd() / file_name}")

# Print the file path(s) from the messages
for file_path_annotation in messages.file_path_annotations:
    print(f"File Paths:")
    print(f"Type: {file_path_annotation.type}")
    print(f"Text: {file_path_annotation.text}")
    print(f"File ID: {file_path_annotation.file_path.file_id}")
    print(f"Start Index: {file_path_annotation.start_index}")
    print(f"End Index: {file_path_annotation.end_index}")
    project_client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=Path(file_path_annotation.text).name)



Messages: {'object': 'list', 'data': [{'id': 'msg_vkBbfdeJSPqOCcFgBPKAHt45', 'object': 'thread.message', 'created_at': 1735823175, 'assistant_id': 'asst_xWUoG3tK8Nx6W5DH7OIGzO31', 'thread_id': 'thread_kJTiCE09WPCtv2gmVgrWLFXz', 'run_id': 'run_x4X6S3O8G54XKT92T3M706Zp', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': 'I have created the bar chart for the operating profit of the companies. You can download the chart using the link below:\n\n[Download Operating Profit Chart](sandbox:/mnt/data/operating_profit_chart.png)', 'annotations': []}}], 'attachments': [], 'metadata': {}}, {'id': 'msg_YgsTtIDFbMIQ4jQoL5eTMLuO', 'object': 'thread.message', 'created_at': 1735823166, 'assistant_id': None, 'thread_id': 'thread_kJTiCE09WPCtv2gmVgrWLFXz', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': 'Could you please create a bar chart for the operating profit using the following data and provide the file to me? Company A: $1.2 million, Company B: $2.5

In [13]:
# Delete the agent once done
project_client.agents.delete_agent(agent.id)
print("Deleted agent")
project_client.__exit__(None, None, None)

Deleted agent


# Bing grounding

In [83]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import BingGroundingTool


# Create an Azure AI Client from a connection string, copied from your Azure AI Foundry project.
# At the moment, it should be in the format "<HostName>;<AzureSubscriptionId>;<ResourceGroup>;<HubName>"
# Customer needs to login to Azure subscription via Azure CLI and set the environment variables

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)

In [84]:
bing_connection = project_client.connections.get(
    connection_name=os.environ["BING_CONNECTION_NAME"]
)
conn_id = bing_connection.id

print(conn_id)

# Initialize agent bing tool and add the connection id
bing = BingGroundingTool(connection_id=conn_id)

# Create agent with the bing tool and process assistant run
project_client.__enter__()
agent = project_client.agents.create_agent(
    model="gpt-4-turbo",
    name="my-assistant",
    instructions="You are a helpful assistant",
    tools=bing.definitions,
    headers={"x-ms-enable-preview": "true"}
)
print(f"Created agent, ID: {agent.id}")

/subscriptions/3490bb6d-9e8c-46c5-b635-f8fea8df3d69/resourceGroups/agent-quickstart-rg2/providers/Microsoft.MachineLearningServices/workspaces/project-demo-agents-oaic/connections/bing_grounding
Created agent, ID: asst_nxhmCf1yRG4IRmSE0tC9QnPp


In [85]:
# Create thread for communication
thread = project_client.agents.create_thread()
print(f"Created thread, ID: {thread.id}")

# Create message to thread
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="What is the top news today",
)
print(f"Created message, ID: {message.id}")

Created thread, ID: thread_0JMs1mv7qtIgjpyWunjS8Ngq
Created message, ID: msg_U7clNkJS5YDxMoQlBtBJ17ak


In [86]:
# Create and process agent run in thread with tools
run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
print(f"Run finished with status: {run.status}")
print(f"Run ID: {run.id}")

# Retrieve run step details to get Bing Search query link
# To render the webpage, we recommend you replace the endpoint of Bing search query URLs with `www.bing.com` and your Bing search query URL would look like "https://www.bing.com/search?q={search query}"
run_steps = project_client.agents.list_run_steps(run_id=run.id, thread_id=thread.id)
run_steps_data = run_steps['data']
print(f"Last run step detail: {run_steps_data}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")


messages = project_client.agents.list_messages(thread_id=thread.id)
print(f"Messages: {messages}")


Run finished with status: completed
Run ID: run_1eVVeN0kRlFprsCJL6q1MSK4
Last run step detail: [{'id': 'step_kvXO6Td9KLaeMgtszHveZsSR', 'object': 'thread.run.step', 'created_at': 1735911094, 'run_id': 'run_1eVVeN0kRlFprsCJL6q1MSK4', 'assistant_id': 'asst_nxhmCf1yRG4IRmSE0tC9QnPp', 'thread_id': 'thread_0JMs1mv7qtIgjpyWunjS8Ngq', 'type': 'message_creation', 'status': 'completed', 'cancelled_at': None, 'completed_at': 1735911099, 'expires_at': None, 'failed_at': None, 'last_error': None, 'step_details': {'type': 'message_creation', 'message_creation': {'message_id': 'msg_0lZWTBxG3x0z2Y8GzKTWQO1L'}}, 'usage': {'prompt_tokens': 1810, 'completion_tokens': 196, 'total_tokens': 2006}}, {'id': 'step_DhIxj0nQC7wTkC9MZJuQgxtw', 'object': 'thread.run.step', 'created_at': 1735911091, 'run_id': 'run_1eVVeN0kRlFprsCJL6q1MSK4', 'assistant_id': 'asst_nxhmCf1yRG4IRmSE0tC9QnPp', 'thread_id': 'thread_0JMs1mv7qtIgjpyWunjS8Ngq', 'type': 'tool_calls', 'status': 'completed', 'cancelled_at': None, 'completed_a

In [74]:
# Delete the agent once done
project_client.agents.delete_agent(agent.id)
print("Deleted agent")
project_client.__exit__(None, None, None)

Deleted agent
